In [92]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math

In [93]:
dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
dataset = dataset.sample(frac=1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test = y[(datasize*7)//10:,:]


In [94]:
def feature_scaling(X_train,y_train):
    training_size = X_train.shape[0]
    X_mean = np.sum(X_train,axis=0) / training_size
    X_var = np.sqrt(np.sum((np.square(X-X_mean)),axis=0)/training_size)
    X_train_reg = (X_train - X_mean) / X_var
    # y_mean = np.sum(y_train,axis=0) / training_size
    # y_var = np.sqrt(np.sum((np.square(y-y_mean)),axis=0)/training_size)
    # y_train_reg = (y_train - y_mean) / y_var

    # return (X_mean,X_var,X_train_reg,y_mean,y_var,y_train_reg)
    return (X_mean,X_var,X_train_reg)


In [103]:
def parsen_window(X_train,y_train,X_test,y_test,window_size):

    test_size = X_test.shape[0]
    train_size = X_train.shape[0]
    # X_m,X_v,X_r,y_m,y_v,y_r = feature_scaling(X_train,y_train)
    X_m,X_v,X_r = feature_scaling(X_train,y_train)
    X_test_reg = (X_test-X_m)/X_v
    # y_test_reg = (y_test-y_m)/y_v
    y_pred = np.array([])
    # print(y_pred)
    for i in range(test_size):
        sample_X = X_test_reg[i:i+1,:]
        sample_y = y_test[i,:]
        dist_vec = np.sqrt(np.sum(np.square(X_r-sample_X),axis=1))
        dist_vec = dist_vec.reshape((train_size,1))
        # print(dist_vec.shape)
        # print(y_train.shape)
        hyp_cube = y_train[dist_vec[:,0]<=window_size,0]
        num1 = np.sum(hyp_cube,axis=0)
        num0 = hyp_cube.shape[0] - num1
        y_pred = np.append(y_pred,[int(num1>=num0)])
        # print(y_pred)
        # if(y_pred==0):
        #     print('yes')
        # break
    return y_pred
        



In [116]:
def accuracy(y_pred,y_test):

    test_size = y_test.shape[0]

    y_pred = y_pred.reshape((y_pred.shape[0],1))

    print(y_test.shape)
    print(y_pred.shape)

    tp = np.sum((y_pred+y_test)==2 , axis=0)[0]
    tn = np.sum(y_pred==y_test , axis=0)[0] - tp
    fp = np.sum(y_pred , axis=0)[0]-tp
    fn = test_size-tp-tn-fp


    print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

    acc = (tp+tn)/test_size
    prec = (tp)/(tp+fp)
    recl = (tp)/(tp+fn)
    f1 = 2*prec*recl/(prec+recl)

    print('Accuracy: {}'.format( acc  ))
    print('Precision: {}'.format( prec  ))
    print('Recall: {}'.format( recl  ))
    print('F1 score: {}'.format( f1  ))

In [126]:
y_pred = parsen_window(X_train,y_train,X_test,y_test,0.6)
accuracy(y_pred,y_test)

(210, 1)
(210, 1)
tp: 84 , tn: 101 , fp: 15.0 , fn: 10.0
Accuracy: 0.8809523809523809
Precision: 0.8484848484848485
Recall: 0.8936170212765957
F1 score: 0.8704663212435233
